In [9]:
import csv
import os
import pandas as pd
import json

移动文件的代码

In [7]:
import os
import shutil

# 源文件夹路径
source_folder = r'E:\ZR\data_split\train\H'
# 目标文件夹路径
target_folder = r'E:\ZR\data_split\val\H'

# 获取源文件夹中所有文件的列表
file_list = os.listdir(source_folder)

# 计算每十个文件中需要移动的文件数量
num_files_to_move = 2

# 初始化一个列表来存储要移动的文件
files_to_move = []

# 遍历文件列表，每十个文件中移动最后的两个文件
for i in range(0, len(file_list), 10):
    # 获取最后的两个文件并添加到要移动的文件列表中
    files_to_move.extend(file_list[i + 8:i + 10])

# 移动文件到目标文件夹
for file_name in files_to_move:
    source_path = os.path.join(source_folder, file_name)
    target_path = os.path.join(target_folder, file_name)
    shutil.move(source_path, target_path)

print(f'{num_files_to_move} 个文件已经从 {source_folder} 移动到 {target_folder}。')

2 个文件已经从 E:\ZR\data_split\train\H 移动到 E:\ZR\data_split\val\H。


将csv转成txt的代码

In [31]:
# 读取CSV文件
M = '3C90'
csv_file = f'./MagNet_data/{M}/merged_file.csv'
output_folder = r'E:\ZR\data_split\merged'  # 替换成你想要保存TXT文件的文件夹路径

# 确保输出文件夹存在，如果不存在则创建它
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 从文件路径中提取材料名称（假设材料名称位于路径的一部分）
material_name = os.path.basename(os.path.dirname(csv_file))

with open(csv_file, 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, )
    # header = next(csvreader)  # 如果CSV文件有标题行，请跳过它

    for line_number, row in enumerate(csvreader, start=1):
        # 生成TXT文件名，格式为CSV文件名（不包含扩展名）+材料名称+行号
        csv_filename = os.path.splitext(os.path.basename(csv_file))[0]
        txt_filename = f"{csv_filename}_{material_name}_{line_number}.txt"
        txt_filepath = os.path.join(output_folder, txt_filename)

        # 将行内容保存到TXT文件中
        with open(txt_filepath, 'w', encoding='utf-8') as txtfile:
            txtfile.write(','.join(row))

print("已经成功将每一行保存为TXT文件，文件名包含CSV文件名、材料名称和行号！")


已经成功将每一行保存为TXT文件，文件名包含CSV文件名、材料名称和行号！


在把csv转成txt之前，先需要把几个csv文件合并成一个

In [21]:
M = 'N49'
csv_file_0 = pd.read_csv(f'./MagNet_data/{M}/B_waveform[T].csv')
csv_file_1 = pd.read_csv(f'./MagNet_data/{M}/Frequency[Hz].csv')
csv_file_2 = pd.read_csv(f'./MagNet_data/{M}/Temperature[C].csv')
csv_file_3 = pd.read_csv(f'./MagNet_data/{M}/Volumetric_losses[Wm-3].csv')
result = pd.concat([csv_file_0, csv_file_1, csv_file_2, csv_file_3], axis=1)
result.to_csv(f'./MagNet_data/{M}/merged_file.csv', index=False)

计算标准化参数

In [79]:
import torch
Norm_Dic = {}

M = 'N87'

in_B = pd.read_csv(f'./MagNet_data/{M}/B_waveform[T].csv')
in_B = torch.from_numpy(in_B.values).float().unsqueeze(2)

in_T = pd.read_csv(f"./MagNet_data/{M}/Temperature[C].csv")
in_T = torch.from_numpy(in_T.values).float().view(-1, 1)

in_F = pd.read_csv(f"./MagNet_data/{M}/Frequency[Hz].csv")
in_F = torch.from_numpy(in_F.values).float().view(-1, 1)

gt_P = pd.read_csv(f"./MagNet_data/{M}/Volumetric_losses[Wm-3].csv")
gt_P = torch.from_numpy(gt_P.values).float().view(-1, 1)

# Transform
in_F = torch.log(in_F)
out_P = torch.log(gt_P)

normB = [torch.mean(in_B), torch.std(in_B)]
normF = [torch.mean(in_F), torch.std(in_F)]
normT = [torch.mean(in_T), torch.std(in_T)]
normP = [torch.mean(out_P), torch.std(out_P)]

Norm_Dic[M] = [normB, normF, normT, normP]

In [81]:
serializable_dict = {}
for key, value in Norm_Dic.items():
    serializable_value = [[item.numpy().tolist() for item in sublist] for sublist in value]
    serializable_dict[key] = serializable_value

In [82]:
with open('Norm_dic.json', 'w') as json_file:
    json.dump(serializable_dict, json_file)

In [50]:
import json
with open("Norm_dic.json", "w") as f:
    f.write(json.dumps(Norm_Dic))

TypeError: Object of type Tensor is not JSON serializable